----
# Visualizer
----

## Use this notebook to visualize .stcs
----
----


In [ ]:
import mne
import os
import glob
import pandas as pd
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import ipywidgets as widgets

derivatives_dir = os.path.expanduser("~/MEG/playground/BIDS_root/derivatives")
subjectlist = glob.glob(os.path.join(derivatives_dir, "*"))
subjectlist = [os.path.basename(f) for f in subjectlist if os.path.isdir(f)]

subject_dropdown = widgets.Dropdown(
options=subjectlist,
value=subjectlist[0],
description='-->',
disabled=False
)
print("Please choose subject to show:")
subject_dropdown

In [ ]:
subject = subject_dropdown.value
base_dir = os.path.join(derivatives_dir, subject)
subjects_dir = os.path.join(base_dir, "freesurfer")
eventfolders = glob.glob(base_dir + "/spikes/*")
eventnames = list(map(lambda e: e.split("/")[-1], eventfolders))
modalities = ["eLORETA_with_peaks", "dSPM"]
eventdict = dict()

for ef in eventfolders:
    e = ef.split("/")[-1]
    stcs = glob.glob(ef +"/stc_*")
    mod_and_file = dict()
    for s in stcs:
        for m in modalities:
            if m in s:
                mod_and_file[m] = s
        eventdict[e] = mod_and_file

df = pd.DataFrame.from_dict(eventdict, orient='index')

times = np.linspace(-20, 5, 6)
time_dict = {
            -20.0: 'a-m20',
            -15.0: 'b-m15',
            -10.0: 'c-m10',
            -5.0 : 'd-m5',
            -0.0 : 'e-m0',
             5.0 : 'f-p5'}

def save_ts():
    surfer_kwargs = dict(
                        hemi=hemi.value, subjects_dir=subjects_dir,
                        clim=dict(kind='value', lims=[low.value, med.value, high.value]),
                        views=views.value,
                        time_unit='ms', size=(800, 800),
                        smoothing_steps=7)
    brains = dict()
    save_directory = os.path.join(base_dir, "spikes", event_number.value, "custom_time_series")
    stc = mne.read_source_estimate(df.loc[event_number.value][modality.value], subject=subject)

    for time in times:
        brains[time] = stc.plot(initial_time=time, **surfer_kwargs)
        text_to_add = str((df.loc[event_number.value][modality.value]).split("/")[-1])
        brain_label = str(text_to_add + ' @ ' + str(time) + 'ms')
        brains[time].add_text(0.1, 0.9, brain_label, 'title', font_size=10)
        custom_pic_name = (event_number.value + '-' + modality.value + '-at-' + time_dict[time] + '.png')
        custom_pic = os.path.join(save_directory, custom_pic_name)
        brains[time].save_image(custom_pic)
        brains[time].close()

def show_stc():
    stc = mne.read_source_estimate(df.loc[event_number.value][modality.value], subject=subject)

    surfer_kwargs = dict(
        hemi=hemi.value, subjects_dir=subjects_dir,
        clim=dict(kind='percent', lims=[85, 97, 99]), 
        #clim=dict(kind='value', lims=[1e-12, 4.5e-12, 8e-12]),
        views=views.value,
        time_unit='ms', size=(1100, 800),
        smoothing_steps=7)

    brain = stc.plot(initial_time=0., **surfer_kwargs)
    file_name = df.loc[event_number.value][modality.value]
    file_name = file_name.split('-rh')[0]
    file_name = file_name.split('-lh')[0]
    file_name = file_name.split("/")[-1]
    text_to_add = str(file_name)
    brain.add_text(0.1, 0.9, text_to_add, 'title', font_size=10)

# Control panel
style = {'description_width': 'initial'}

event_number = widgets.Dropdown(
    options=df.index.values,
    value=df.index.values[0],
    description='Event:',
    disabled=False)

modality = widgets.Dropdown(
    options=df.columns,
    value=df.columns[0],
    description='Modalität:',
    disabled=False)

hemi = widgets.Dropdown(
    options=['lh', 'rh', 'split', 'both'],
    value='lh',
    description='Hemi:',
    disabled=False)

views = widgets.Dropdown(
    options=["lateral", "medial", "rostral", "caudal", "ventral", "dorsal"],
    value="lateral",
    description='Ansicht:',
    disabled=False)

b_show_stc = widgets.Button(
    description="show viz",
    disabled=True,
    button_style="success",)
    #icon='check')
b_show_stc.on_click(show_stc)

b_save_ts = widgets.Button(
    description="save time series",
    disabled=True,
    button_style="info",)
    #icon="fas fa-arrow-alt-circle-down",)
    #icon="check")
b_save_ts.on_click(save_ts)

high = widgets.FloatSlider(
    value=7,
    min=1,
    max=100,
    step=.5,
    description='high (*10^-13):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style=style)

med = widgets.FloatSlider(
    value=7,
    min=1,
    max=100,
    step=.5,
    description='med (*10^-13):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style=style)

low = widgets.FloatSlider(
    value=7,
    min=1,
    max=100,
    step=.5,
    description='low (*10^-13):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style=style)

# Choosing widget
left = widgets.VBox([event_number, modality, hemi, views])
right = widgets.VBox([high, med, low, widgets.HBox([b_show_stc, b_save_ts])])

widgets.HBox([left, right])

## Show time series:

In [ ]:
show_stc()

## Save time series:

In [ ]:
save_ts()

## Alternative if something fails: Manual configuration
### Use the table from below, line-number = event-number (index starts at 0)

In [37]:
df

,dSPM,eLORETA_with_peaks
Gr_1,/home/idrael/MEG/playground/BIDS_root/derivati...,/home/idrael/MEG/playground/BIDS_root/derivati...
Gr_2,/home/idrael/MEG/playground/BIDS_root/derivati...,/home/idrael/MEG/playground/BIDS_root/derivati...


In [ ]:
event_number    = 0      # Event (= line)
index           = 0      # Modality (= row)


#hemi = 'rh'           #right hemisphere
hemi = 'lh'           #left hemisphere
#hemi = 'split'        #both hemispheres in split view
#hemi = 'both'         #both hemispheres in anatomical view

views = ['lateral']
#views = ['medial']
#views = ['rostral']
#views = ['caudal']
#views = ['ventral']
#views = ['dorsal']
#views = ['lateral', 'medial']   # --> show lateral + medial simultaneously
#views = ['ventral', 'dorsal']
#views = ['lateral', 'medial', 'ventral', 'dorsal']

stc = mne.read_source_estimate(df.iloc[event_number][index], subject=subsubj)

surfer_kwargs = dict(
    hemi=hemi.value, subjects_dir=subjects_dir,
    clim=dict(kind='percent', lims=[85, 97, 99]), 
    #clim=dict(kind='value', lims=[1e-12, 4.5e-12, 8e-12]),
    views=views.value,
    time_unit='ms', size=(1100, 800),
    smoothing_steps=7)

brain = stc.plot(initial_time=0., **surfer_kwargs)
file_name = df.iloc[event_number][index]
file_name = file_name.split('-rh')[0]
file_name = file_name.split('-lh')[0]
file_name = file_name.split("/")[-1]
text_to_add = str(file_name)
brain.add_text(0.1, 0.9, text_to_add, 'title', font_size=10)

## Visualization

In [ ]:
stc = mne.read_source_estimate(df.iloc[event_number][index], subject=subsubj)

surfer_kwargs = dict(
    hemi=hemi.value, subjects_dir=subjects_dir,
    clim=dict(kind='percent', lims=[85, 97, 99]), 
    #clim=dict(kind='value', lims=[1e-12, 4.5e-12, 8e-12]),
    views=views.value,
    time_unit='ms', size=(1100, 800),
    smoothing_steps=7)

brain = stc.plot(initial_time=0., **surfer_kwargs)
file_name = df.iloc[event_number][index]
file_name = file_name.split('-rh')[0]
file_name = file_name.split('-lh')[0]
file_name = file_name.split("/")[-1]
text_to_add = str(file_name)
brain.add_text(0.1, 0.9, text_to_add, 'title', font_size=10)

----
### visualization thresholds

In [ ]:
# --> set via sliders above or uncomment and enter clim thresholds,  1.25e-3  translates to 0.00125
"""
high     = 7e-11
medium   = 5e-11
low      = 1e-11  
"""

## Save custom time series

In [ ]:
surfer_kwargs = dict(
    hemi=hemi, subjects_dir=subjects_dir,
    clim=dict(kind='value', lims=[low, medium, high]),
    views=views,
    time_unit='ms', size=(800, 800),
    smoothing_steps=7)

times = np.linspace(-20, 5, 6)
print(times)

brains = dict()
time_dict = {
            -20.0: 'a-m20',
            -15.0: 'b-m15',
            -10.0: 'c-m10',
            -5.0 : 'd-m5',
            -0.0 : 'e-m0',
             5.0 : 'f-p5'}

event_name = df.index[event_number]
save_directory = os.path.join(base_dir, "spikes", event_name, "custom_time_series")
modality = df.columns[index]

for time in times:
    brains[time] = stc.plot(initial_time=time, **surfer_kwargs)
    brain_label = str(text_to_add + ' @ ' + str(time) + 'ms')
    brains[time].add_text(0.1, 0.9, brain_label, 'title', font_size=10)
    text_to_add = str((df.iloc[event_number][index]).split("/")[-1])
    custom_pic_name = (event_name + '-' + modality + '-at-' + time_dict[time] + '.png')
    custom_pic = os.path.join(save_directory, custom_pic_name)
    brains[time].save_image(custom_pic)
    brains[time].close()